In [236]:
VALVES = {}
CONNECTIONS = {}

def bfs(start, end):
    start_valve = start
    end_valve = [valve for valve in VALVES.values() if valve == end][0]

    visited = {start_valve: None}
    queue = [start_valve]

    loops=0
    while queue:
        loops+=1
        current_valve = queue.pop(0)
        if current_valve == end_valve:
            shortest_path = []
            while current_valve:
                shortest_path.append(current_valve)
                current_valve = visited[current_valve]
            return shortest_path

        for valve in current_valve.connections:
            if valve not in visited:
                visited[valve] = current_valve
                queue.append(valve)

class Valve:
    def __init__(self, name, flow):
        self.name = name
        self.flow = flow
        self.connections = []
        self.valve_distances = {}
    
    def add_connection(self,other):
        self.connections.append(other)
    
    def calculate_distances(self):
        for valve in VALVES.values():
            if valve == self:
                self.valve_distances[valve]=1
            dis_calculated = valve.valve_distances.get(self,None)
            if dis_calculated:
                self.valve_distances[valve]=dis_calculated
                continue
            distance = len(bfs(self, valve))
            self.valve_distances[valve]=distance

    def __repr__(self):
        return f"Valve({self.name}, {self.flow})"

with open("16.txt") as i:
    q_input = i.read().splitlines()

import re
for line in q_input:
    line_vals = re.findall(r'[A-Z][A-Z]|\d\d|\d', line)
    valve_name = line_vals.pop(0)
    flow_rate = int(line_vals.pop(0))
    valve=Valve(valve_name, flow_rate)
    VALVES[valve_name]=valve
    CONNECTIONS[valve] = line_vals

for valve, connections in CONNECTIONS.items():
    for connection in connections:
        valve.add_connection(VALVES[connection])

for valve in VALVES.values():
    valve.calculate_distances()

TARGET_VALVES = [valve for valve in VALVES.values() if valve.flow > 0]


best_score = 0
left_off_valves = None

def find_scores(current_valve: Valve, score, time_left, off_valves, activate):
    global best_score
    global left_off_valves
    if time_left == 0:
        score += current_valve.flow*time_left
        if score > best_score:
            best_score = score
            left_off_valves = off_valves
        return
    elif time_left < 0:
        if score > best_score:
            best_score = score
            left_off_valves = off_valves
        return
    elif not off_valves:
        score += current_valve.flow*time_left
        if score > best_score:
            best_score = score
            left_off_valves = off_valves
        return
    if activate:
        score += current_valve.flow*time_left
    for new_valve in off_valves:
        distance = current_valve.valve_distances[new_valve]
        remaining_valves = [valve for valve in off_valves if valve != new_valve]
        find_scores(new_valve, score, time_left-distance, remaining_valves, True)

find_scores(VALVES["AA"], 0, 30, TARGET_VALVES, False)

print("Part 1:", best_score)

Part 1: 1376


In [235]:
best_score = 0
best_scores = []
left_off_valves = None

def find_scores(current_valve: Valve, score, time_left, off_valves, activate, is_elephant=False):
    global best_score
    global left_off_valves
    if time_left == 0:
        score += current_valve.flow*time_left
        if score > best_score:
            best_score = score
            left_off_valves = off_valves
        if score > 1020 and not is_elephant:
            find_scores(VALVES["AA"], score, 26, off_valves, False, True)
        return
    elif time_left < 0:
        if score > best_score:
            best_score = score
            left_off_valves = off_valves
        if score > 1020 and not is_elephant:
            find_scores(VALVES["AA"], score, 26, off_valves, False, True)
        return
    elif not off_valves:
        score += current_valve.flow*time_left
        if score > best_score:
            best_score = score
            left_off_valves = off_valves
        return
    if activate:
        score += current_valve.flow*time_left
    for new_valve in off_valves:
        distance = current_valve.valve_distances[new_valve]
        remaining_valves = [valve for valve in off_valves if valve != new_valve]
        find_scores(new_valve, score, time_left-distance, remaining_valves, True, is_elephant)

find_scores(VALVES["AA"], 0, 26, TARGET_VALVES, False)

print("Part 2:", best_score)

Part 2: 1933
